As per Task 2, we need to utilize the Foursuare location data, to get the latitude and the logitude coordinates of each neighborhood. For this we will be using the Geocoder Python Package.

Task 2: 
        Add the latitude and longitude coordinates to the dataframe.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
# download url data from internet
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
Canada_data = BeautifulSoup(source, 'lxml')

Creat a new Dataframe

In [2]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

In [27]:
for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

Clean dataframe 

In [28]:
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1

In [29]:
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [30]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))

In [31]:
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')
df2.rename(columns={'Postalcode':'Postal Code'}, inplace=True)
df2.head()

,Postal Code,Borough,Neighborhood
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,"M9AEtobicoke(Islington Avenue), M9AEtobicoke(I..."
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,M9BEtobicoke(West Deane Park / Princess Garden...
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,"M9ENot assigned, M9ENot assigned"
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,"M9GNot assigned, M9GNot assigned"


In [36]:
# Cleaning the table by removing "\n" from each column.
df2["Postal Code"] = df2["Postal Code"].str.replace("\n","")
df2["Borough"] = df2["Borough"].str.replace("\n","")
df2["Neighborhood"] = df2["Neighborhood"].str.replace("\n","")
df2.head()

,Postal Code,Borough,Neighborhood
0,M1ANot assigned,M2ANot assigned,"M9AEtobicoke(Islington Avenue), M9AEtobicoke(I..."
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,"M9ENot assigned, M9ENot assigned"
4,M1GScarborough(Woburn),M2GNot assigned,"M9GNot assigned, M9GNot assigned"


In [46]:
df2 = pd.DataFrame({'Postal Code':['M1A', 'M1B', 'M1C', 'M1E', 'M1G'],
                   'Borough':[' M2A', '  M2B', 'M2C', 'M2E', 'M2G'],
                   'Neighborhood':['M9A', 'M9B', 'M9C', 'M9E', 'M9G']
                  })
df2.head()

,Postal Code,Borough,Neighborhood
0,M1A,M2A,M9A
1,M1B,M2B,M9B
2,M1C,M2C,M9C
3,M1E,M2E,M9E
4,M1G,M2G,M9G


For this task, we just have to use a prepared csv to retrieve the coordinates. Extract csv with Toronto geographical coordinates to dataframe.

In [48]:
toronto_geocsv = 'https://cocl.us/Geospatial_data'
!wget -q -O 'toronto_m.geospatial_data.csv' toronto_geocsv
geocsv_data = pd.read_csv(toronto_geocsv).set_index("Postal Code")
geocsv_data.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


Let's now combine the two

In [49]:
df = pd.merge(geocsv_data, df2, on='Postal Code')
df.head()

,Postal Code,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,M2B,M9B
1,M1C,43.784535,-79.160497,M2C,M9C
2,M1E,43.763573,-79.188711,M2E,M9E
3,M1G,43.770992,-79.216917,M2G,M9G


In [50]:
df.to_csv('Toronto.TASK_II_df.csv',index=False)

In [51]:
df.shape

(4, 5)